# Preprocessing

앞에서 분류한 것을 바탕으로, one-hot / scaling 등 실제 계산 진행

## 불러오기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm;
import sklearn
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
train =  pd.read_csv('data/EDA/train_EDA_0910.csv')
test =  pd.read_csv('data/EDA/test_EDA_0910.csv')
feat_class = pd.read_csv('data/EDA/feat_class_0910.csv')

In [ ]:
feat_raw = feat_class['raw'].dropna().values.tolist()
feat_dis = feat_class['dis'].dropna().values.tolist()
feat_map = feat_class['map'].dropna().values.tolist()
feat_onehot = feat_class['onehot'].dropna().values.tolist()
feat_extra = feat_class['extra'].dropna().values.tolist()
feature_del = feat_class['del'].dropna().values.tolist()

# 분류별

- 분류별 feature들을 불러와서 각 분류에 맞게 핸들링

### Raw

In [ ]:
feat_raw

### discrete

In [ ]:
feat_dis

### map

In [ ]:
feat_map

### onehot

In [ ]:
feat_onehot

In [ ]:
# feature_map도 전부 onehot으로.
#feat_onehot.extend(feat_map)

In [ ]:
train = pd.get_dummies(train, columns = feat_onehot)
test = pd.get_dummies(test, columns = feat_onehot)

### extra

In [ ]:
feat_extra

neighbor, MoSold는 일단 원핫.

In [ ]:
train = pd.get_dummies(train, columns = ['Neighborhood', 'MoSold'])
test = pd.get_dummies(test, columns = ['Neighborhood', 'MoSold'])

- 년도는 절대적인 년도보다 지어진지 얼마나 된 건물을 샀는지가 더 중요해보여서 지어진 년도 - 팔린 년도 로 계산
- 차고가 지어진 년도는 모르겟어서 일단 제거..

In [ ]:
train['YearOld'] = (train['YearBuilt'] + train['YearRemodAdd'] )/2
train['YearOld'] -= train['YrSold']
test['YearOld'] = (test['YearBuilt'] + test['YearRemodAdd'] )/2
test['YearOld'] -= test['YrSold']

In [ ]:
#Extra에서 따로 계산하고 지워줘야 할 컬럼 추가
feature_del.extend(['YearBuilt', 'YearRemodAdd',
                    'Exterior1st', 'Exterior2nd', 'YrSold', 'GarageYrBlt'])

## Delete

In [ ]:
feature_del

In [ ]:
train = train.drop(columns = feature_del)
test = test.drop(columns = feature_del)

One-hot 과정에서 train / test 둘 중 하나에만 존재하는 컬럼 제거

In [ ]:
train_comp = list(set(train) - set(test)-{'SalePrice'})
test_comp = list(set(test)- set(train))
train = train.drop(columns = train_comp)
test = test.drop(columns = test_comp)

개수가 20개 이하인 one-hot column들 제거함.

In [ ]:
small_samples = []
for feat in train.columns.to_list():
    if abs(train[feat]).sum() < 20:
        small_samples.append(feat)
len(small_samples)

In [ ]:
train = train.drop(columns = small_samples)
test = test.drop(columns = small_samples)

## Min-Max Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()

In [ ]:
train_X = train.drop('SalePrice', axis=1)
train_y = train['SalePrice']

In [ ]:
X_scale = minmax_scaler.fit_transform(train_X)
train = pd.DataFrame(X_scale, columns=train_X.columns, index=list(train_X.index.values))
test_scale = minmax_scaler.fit_transform(test)
test = pd.DataFrame(test_scale, columns=test.columns, index=list(test.index.values))

In [ ]:
train_y = np.log1p(train_y)

In [ ]:
train['SalePrice'] = train_y
train

In [ ]:
train.to_csv('data/preprocess/train_0910.csv', index = False)
test.to_csv('data/preprocess/test_0910.csv', index = False)